<br>
<h1 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f7 ; color : #fe346e; text-align: center; border-radius: 100px 85;">CommonLit Readability<br> BERT + Roberta + Model Explainabilty</h1>
<br>

In [ ]:
!pip install -q nlpretext loguru

# Required Data Loading and Util libraries 

In [ ]:
import os
import gc
import copy
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from tqdm import tqdm
from collections import defaultdict

from loguru import logger


import warnings
warnings.filterwarnings("ignore")

# Required Modeling Libraries

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

import transformers
from transformers import BertTokenizer, BertModel, BertConfig
from transformers import AdamW, get_linear_schedule_with_warmup

from sklearn.model_selection import StratifiedKFold, KFold
from scipy.stats import ks_2samp

# NLP Preprocessing Libraries

In [ ]:
from nlpretext import Preprocessor
from nlpretext.basic.preprocess import (normalize_whitespace, remove_punct, 
                                        remove_eol_characters, remove_stopwords, 
                                        lower_text, unpack_english_contractions)

from nlpretext.social.preprocess import remove_html_tags

# 2. Data Loading

In [ ]:
%%time
df_train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
df_test = pd.read_csv("../input/commonlitreadabilityprize/test.csv")

In [ ]:
df_train.info()

# 3. Data Wrangling or Preprocessing

In [ ]:
# Sample Text before preprocessing

df_train['excerpt'][0]

## To know more details about nlpretext library 

[Link](https://medium.com/artefact-engineering-and-data-science/introducing-nlpretext-a8bb7c03df89)

In [ ]:
preprocessor = Preprocessor()
preprocessor.pipe(unpack_english_contractions)
preprocessor.pipe(remove_eol_characters)
preprocessor.pipe(lower_text)
preprocessor.pipe(normalize_whitespace)
preprocessor.pipe(remove_punct)
#preprocessor.pipe(remove_stopwords(lang='en'))

In [ ]:
df_train['excerpt'] = df_train['excerpt'].apply(preprocessor.run)

In [ ]:
df_test['excerpt'] = df_test['excerpt'].apply(preprocessor.run)

In [ ]:
# Sample Text after preprocessing

df_train['excerpt'][0]

In [ ]:
def max_words(df):
    lengths = df['excerpt'].apply(lambda x: len(x.split()))
    return max(lengths)

In [ ]:
excerpt_lenghts = df_train['excerpt'].apply(lambda x: len(x.split()))
sns.histplot(excerpt_lenghts)

In [ ]:
max(excerpt_lenghts)

### Observation :- 
    For BERT , Its required to share the maximum words in the text 

In [ ]:
class CFG:
    
    max_len = 218
    
    train_batch_size = 8
    valid_batch_size = 16
    
    epochs = 5
    learning_rate = 1e-5
    n_accumulate = 1
    folds = 5
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Sampling Strategy - KFOLD

In [ ]:
def create_folds(df, n_s=5, n_grp=None):
    df['Fold'] = -1
    
    if n_grp is None:
        skf = KFold(n_splits=n_s)
        target = df.target
    else:
        skf = StratifiedKFold(n_splits=n_s)
        df['grp'] = pd.cut(df.target, n_grp, labels=False)
        target = df.grp
    
    for fold_no, (t, v) in enumerate(skf.split(target, target)):
        df.loc[v, 'Fold'] = fold_no
    return df

In [ ]:
df_train = create_folds(df_train, n_s=CFG.folds, n_grp=100)
df_train['Fold'].value_counts()

## Let See How the Taget is distribured accross folds

In [ ]:
def viz_tgt_folds(df):
    fig, axs = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(10,4))
    for i, ax in enumerate(axs):
        ax.hist(df[df.Fold == i]['target'], bins=100, density=True, label=f'Fold-{i}')
        if i == 0:
            ax.set_ylabel('Frequency')
        if i == 2:
            ax.set_xlabel("Target")
        ax.legend(frameon=False, handlelength=0)
    plt.tight_layout()
    plt.show()
    
    return None

In [ ]:
viz_tgt_folds(df)

### Observation :- 
#### Target distribution accross the folds looks normal and evenly distributed

## Ensuring the distribution of Folds

We can compare any two folds with the **Kolmogorov-Smirnov** test to examine if the folds come from the same distribution. Let's compare all folds with the 1st fold for simplicity. The test results are given below. Indeed, the low KS (~0.0008) and high probability (1.0) values confirm that all folds come from the same distribution.

In [ ]:
for fold in np.sort(df.Fold.unique())[1:]:
    print(f'Fold 0 vs {fold}:', ks_2samp(df.loc[df.Fold==0,'target'], df.loc[df.Fold==fold,'target']))

### Observation :- 
#### We Can Statisticaly say that target distribution even where as PValue approaching 1 and Statistic is less than zero

<br>
<h1 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f7 ; color : #fe346e; text-align: center; border-radius: 100px 85;">CommonLit Readability<br> BERT Model</h1>
<br>

## Preparing Torch dataset for BERT 

In [ ]:
class BERTDset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.text = df['excerpt'].values
        self.target = df['target'].values
        self.max_len = max_len
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(self.target[index], dtype=torch.float)
        }

## Loss Function

In [ ]:
def eval_metrics(outputs, targets):
    return torch.sqrt(nn.MSELoss()(outputs, targets))

In [ ]:
class BERTClass(nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(768, 1)
    
    def forward(self, ids, mask, token_type_ids):
        _, output = self.bert(ids, attention_mask = mask, 
                              token_type_ids = token_type_ids, 
                              return_dict=False)
        output = self.dropout(output)
        output = self.fc(output)
        return output

model = BERTClass()
model.to(CFG.device);

<br>
<h1 style = "font-size:60px; font-family:Garamond ; font-weight : normal; background-color: #f6f5f7 ; color : #fe346e; text-align: center; border-radius: 100px 85;">BERT Model - Training & Validation </h1>
<br>

In [ ]:
def training(model,optimizer,scheduler,dataloader,device,epoch):
    model.train()
    scaler = amp.GradScaler()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype = torch.float)
        
        batch_size = ids.size(0)
        
        with amp.autocast(enabled=True):
            outputs = model(ids, mask, token_type_ids)
            loss = eval_metrics(outputs, targets)
            loss = loss / CFG.n_accumulate
            
        scaler.scale(loss).backward()
        
        if (step + 1) % CFG.n_accumulate == 0:
            scaler.step(optimizer)
            scaler.update()
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss/dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [ ]:
def inferance(model,optimizer,scheduler,dataloader,device,epoch):
    
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype = torch.float)
        
        batch_size = ids.size(0)
        
        outputs = model(ids, mask, token_type_ids)
        loss = eval_metrics(outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss/dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    gc.collect()
    
    return epoch_loss

In [ ]:
@logger.catch
def run(model, optimizer, scheduler, device, num_epochs):    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = training(model, optimizer, scheduler, dataloader=train_loader, 
                                           device=CFG.device, epoch=epoch)
        
        valid_epoch_loss = inferance(model, optimizer, scheduler, dataloader=valid_loader, 
                                           device=CFG.device, epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(valid_epoch_loss)
        
        # deep copy the model
        if valid_epoch_loss <= best_loss:
            print(f"Validation Loss Improved ({best_loss} ---> {valid_epoch_loss})")
            best_loss = valid_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = "Loss{:.4f}_epoch{:.0f}.bin".format(best_loss, epoch)
            torch.save(model.state_dict(), PATH)
            print("Model Saved")
    
    print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [ ]:
def prepare_data(df,fold):
    
    df_train = df[df.Fold == fold].reset_index(drop=True)
    df_valid = df[df.Fold == fold].reset_index(drop=True)
    
    train_dataset = BERTDset(df_train, CFG.tokenizer, CFG.max_len)
    valid_dataset = BERTDset(df_valid, CFG.tokenizer, CFG.max_len)

    train_loader = DataLoader(train_dataset, batch_size=CFG.train_batch_size, 
                              num_workers=4, shuffle=True, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.valid_batch_size, 
                              num_workers=4, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [ ]:
train_loader, valid_loader = prepare_data(df_train,fold=1)

In [ ]:
for x in valid_loader:
    print(x)
    break

In [ ]:
# Defining Optimizer with weight decay to params other than bias and layer norms
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
     'weight_decay': 0.0001},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
     'weight_decay': 0.0}
    ]  

optimizer = AdamW(optimizer_parameters, lr=CFG.learning_rate)

#Defining LR Scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=len(train_loader)*CFG.epochs
)

In [ ]:
lrs = []
for epoch in range(1, CFG.epochs + 1):
    scheduler.step()
    lrs.append(optimizer.param_groups[0]["lr"])
plt.plot(lrs);

In [ ]:
model, history = run(model, optimizer, scheduler, device=CFG.device, num_epochs=CFG.epochs)

In [ ]:
fig = plt.figure(figsize=(22,8))
plt.plot(history['Train Loss'], label='Train Loss')
plt.plot(history['Valid Loss'], label='Valid Loss')
plt.legend()
plt.title('Loss Curve');

### ... In progress 